In [1]:
import torch
from torchvision import transforms, datasets

In [2]:
# Parameters and DataLoaders
PATH = './model_test_5.pth'
batch_size = 30

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)

cuda:0


In [4]:
data_transform = transforms.Compose([
    transforms.RandomChoice([
        transforms.CenterCrop(200),
        transforms.RandomAffine(30, fillcolor=128),
        transforms.RandomAffine(0,shear=(0,40), fillcolor=128)
    ]),
    transforms.Resize(256),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

trainset = datasets.ImageFolder(root='./train_img', transform=data_transform)
trainset.idx_to_class = {v: k for k, v in trainset.class_to_idx.items()}

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=4)

In [5]:
testset = datasets.ImageFolder(root='./test_img', transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
]))
testset.idx_to_class = {v: k for k, v in testset.class_to_idx.items()}

testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

In [6]:
trainset.idx_to_class

{0: 'aceite',
 1: 'agua',
 2: 'arroz',
 3: 'azucar',
 4: 'cafe',
 5: 'caramelo',
 6: 'cereal',
 7: 'chips',
 8: 'chocolate',
 9: 'especias',
 10: 'frijoles',
 11: 'gaseosa',
 12: 'harina',
 13: 'jamon',
 14: 'jugo',
 15: 'leche',
 16: 'maiz',
 17: 'miel',
 18: 'nueces',
 19: 'pasta',
 20: 'pescado',
 21: 'salsatomate',
 22: 'te',
 23: 'torta',
 24: 'vinagre'}

In [7]:
import torch.nn as nn
import torch.nn.functional as F

net = torch.hub.load('pytorch/vision:v0.5.0', 'inception_v3', pretrained=False, aux_logits=False)
if torch.cuda.device_count() > 1:
  print("Let's use", torch.cuda.device_count(), "GPUs!")
  # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
  # net = nn.DataParallel(net)
net.to(device)
net.train()

Using cache found in /home/lcarranza/.cache/torch/hub/pytorch_vision_v0.5.0


Let's use 2 GPUs!


Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, t

In [11]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.0001)

In [19]:
correctAnt=0
for epoch in range(1000):  # loop over the dataset multiple times

    running_loss = 0.0
    print('Starting Epoch: ',epoch)
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        # inputs, labels = data
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
            
        if i % 96 == 95:    # print every 100 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0
            if epoch>100:
                correct = 0
                with torch.no_grad():
                    for data2 in testloader:
                        # images, labels = data
                        images2, labels2 = data2[0].to(device), data2[1].to(device)
                        outputs = net(images2)
                        _, predicted = torch.max(outputs.data, 1)
                        correct += (predicted == labels2).sum().item()
                    if correct > correctAnt:
                        correctAnt = correct
                        if epoch<400:
                            torch.save(net.state_dict(), PATH)
                            print("Nuevo modelo!")
                        else:
                            torch.save(net.state_dict(), './model_test_'+str(epoch)+'.pth')
                            print("Nuevo modelo!: WOW",'./model_test_'+str(epoch)+'.pth')

print('Finished Training')

Starting Epoch:  0
[1,    96] loss: 1.809
Starting Epoch:  1
[2,    96] loss: 1.708
Starting Epoch:  2
[3,    96] loss: 1.608
Starting Epoch:  3
[4,    96] loss: 1.608
Starting Epoch:  4
[5,    96] loss: 1.560
Starting Epoch:  5
[6,    96] loss: 1.497
Starting Epoch:  6
[7,    96] loss: 1.431
Starting Epoch:  7
[8,    96] loss: 1.403
Starting Epoch:  8
[9,    96] loss: 1.334
Starting Epoch:  9
[10,    96] loss: 1.295
Starting Epoch:  10
[11,    96] loss: 1.259
Starting Epoch:  11
[12,    96] loss: 1.185
Starting Epoch:  12
[13,    96] loss: 1.091
Starting Epoch:  13
[14,    96] loss: 1.107
Starting Epoch:  14
[15,    96] loss: 1.032
Starting Epoch:  15
[16,    96] loss: 0.981
Starting Epoch:  16
[17,    96] loss: 0.981
Starting Epoch:  17
[18,    96] loss: 0.941
Starting Epoch:  18
[19,    96] loss: 0.899
Starting Epoch:  19
[20,    96] loss: 0.876
Starting Epoch:  20
[21,    96] loss: 0.826
Starting Epoch:  21
[22,    96] loss: 0.778
Starting Epoch:  22
[23,    96] loss: 0.738
Startin

Starting Epoch:  182
[183,    96] loss: 0.072
Starting Epoch:  183
[184,    96] loss: 0.059
Starting Epoch:  184
[185,    96] loss: 0.067
Starting Epoch:  185
[186,    96] loss: 0.034
Starting Epoch:  186
[187,    96] loss: 0.045
Starting Epoch:  187
[188,    96] loss: 0.050
Starting Epoch:  188
[189,    96] loss: 0.051
Starting Epoch:  189
[190,    96] loss: 0.059
Starting Epoch:  190
[191,    96] loss: 0.071
Starting Epoch:  191
[192,    96] loss: 0.058
Starting Epoch:  192
[193,    96] loss: 0.061
Starting Epoch:  193
[194,    96] loss: 0.060
Starting Epoch:  194
[195,    96] loss: 0.049
Starting Epoch:  195
[196,    96] loss: 0.042
Starting Epoch:  196
[197,    96] loss: 0.065
Starting Epoch:  197
[198,    96] loss: 0.075
Starting Epoch:  198
[199,    96] loss: 0.068
Starting Epoch:  199
[200,    96] loss: 0.063
Starting Epoch:  200
[201,    96] loss: 0.047
Starting Epoch:  201
[202,    96] loss: 0.035
Starting Epoch:  202
[203,    96] loss: 0.029
Starting Epoch:  203
[204,    96] 

[360,    96] loss: 0.017
Starting Epoch:  360
[361,    96] loss: 0.020
Starting Epoch:  361
[362,    96] loss: 0.024
Starting Epoch:  362
[363,    96] loss: 0.022
Starting Epoch:  363
[364,    96] loss: 0.017
Starting Epoch:  364
[365,    96] loss: 0.014
Starting Epoch:  365
[366,    96] loss: 0.025
Starting Epoch:  366
[367,    96] loss: 0.033
Starting Epoch:  367
[368,    96] loss: 0.032
Starting Epoch:  368
[369,    96] loss: 0.032
Starting Epoch:  369
[370,    96] loss: 0.066
Starting Epoch:  370
[371,    96] loss: 0.055
Starting Epoch:  371
[372,    96] loss: 0.030
Starting Epoch:  372
[373,    96] loss: 0.019
Starting Epoch:  373
[374,    96] loss: 0.030
Starting Epoch:  374
[375,    96] loss: 0.035
Starting Epoch:  375
[376,    96] loss: 0.018
Starting Epoch:  376
[377,    96] loss: 0.016
Starting Epoch:  377
[378,    96] loss: 0.011
Starting Epoch:  378
[379,    96] loss: 0.015
Starting Epoch:  379
[380,    96] loss: 0.038
Starting Epoch:  380
[381,    96] loss: 0.022
Starting 

Starting Epoch:  536
[537,    96] loss: 0.013
Starting Epoch:  537
[538,    96] loss: 0.018
Starting Epoch:  538
[539,    96] loss: 0.031
Starting Epoch:  539
[540,    96] loss: 0.008
Starting Epoch:  540
[541,    96] loss: 0.011
Starting Epoch:  541
[542,    96] loss: 0.012
Starting Epoch:  542
[543,    96] loss: 0.008
Starting Epoch:  543
[544,    96] loss: 0.004
Starting Epoch:  544
[545,    96] loss: 0.008
Starting Epoch:  545
[546,    96] loss: 0.017
Starting Epoch:  546
[547,    96] loss: 0.012
Starting Epoch:  547
[548,    96] loss: 0.036
Starting Epoch:  548
[549,    96] loss: 0.022
Starting Epoch:  549
[550,    96] loss: 0.011
Starting Epoch:  550
[551,    96] loss: 0.006
Starting Epoch:  551
[552,    96] loss: 0.026
Starting Epoch:  552
[553,    96] loss: 0.014
Starting Epoch:  553
[554,    96] loss: 0.030
Starting Epoch:  554
[555,    96] loss: 0.017
Starting Epoch:  555
[556,    96] loss: 0.016
Starting Epoch:  556
[557,    96] loss: 0.026
Starting Epoch:  557
[558,    96] 

[715,    96] loss: 0.008
Starting Epoch:  715
[716,    96] loss: 0.002
Starting Epoch:  716
[717,    96] loss: 0.003
Starting Epoch:  717
[718,    96] loss: 0.004
Starting Epoch:  718
[719,    96] loss: 0.011
Starting Epoch:  719
[720,    96] loss: 0.027
Starting Epoch:  720
[721,    96] loss: 0.031
Starting Epoch:  721
[722,    96] loss: 0.028
Starting Epoch:  722
[723,    96] loss: 0.035
Starting Epoch:  723
[724,    96] loss: 0.011
Starting Epoch:  724
[725,    96] loss: 0.013
Starting Epoch:  725
[726,    96] loss: 0.016
Starting Epoch:  726
[727,    96] loss: 0.020
Starting Epoch:  727
[728,    96] loss: 0.012
Starting Epoch:  728
[729,    96] loss: 0.012
Starting Epoch:  729
[730,    96] loss: 0.005
Starting Epoch:  730
[731,    96] loss: 0.003
Starting Epoch:  731
[732,    96] loss: 0.005
Starting Epoch:  732
[733,    96] loss: 0.011
Starting Epoch:  733
[734,    96] loss: 0.011
Starting Epoch:  734
[735,    96] loss: 0.015
Nuevo modelo!: WOW ./model_test_734.pth
Starting Epoch:

Starting Epoch:  892
[893,    96] loss: 0.018
Starting Epoch:  893
[894,    96] loss: 0.025
Starting Epoch:  894
[895,    96] loss: 0.008
Starting Epoch:  895
[896,    96] loss: 0.019
Starting Epoch:  896
[897,    96] loss: 0.006
Starting Epoch:  897
[898,    96] loss: 0.013
Starting Epoch:  898
[899,    96] loss: 0.005
Starting Epoch:  899
[900,    96] loss: 0.009
Starting Epoch:  900
[901,    96] loss: 0.012
Starting Epoch:  901
[902,    96] loss: 0.007
Starting Epoch:  902
[903,    96] loss: 0.020
Starting Epoch:  903
[904,    96] loss: 0.019
Starting Epoch:  904
[905,    96] loss: 0.031
Starting Epoch:  905
[906,    96] loss: 0.017
Starting Epoch:  906
[907,    96] loss: 0.017
Starting Epoch:  907
[908,    96] loss: 0.008
Starting Epoch:  908
[909,    96] loss: 0.006
Starting Epoch:  909
[910,    96] loss: 0.016
Starting Epoch:  910
[911,    96] loss: 0.004
Starting Epoch:  911
[912,    96] loss: 0.014
Starting Epoch:  912
[913,    96] loss: 0.014
Starting Epoch:  913
[914,    96] 

In [20]:
PATH = './model_test_734.pth'
torch.save(net.state_dict(), PATH)

In [21]:
dataiter = iter(testloader)
images, labels = dataiter.next()
images = images.to(device)
labels = labels.to(device)


In [22]:
net = torch.hub.load('pytorch/vision:v0.5.0', 'inception_v3', pretrained=False, aux_logits=False)
if torch.cuda.device_count() > 1:
  print("Let's use", torch.cuda.device_count(), "GPUs!")
  # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
  # net = nn.DataParallel(net)
net.to(device)
net.load_state_dict(torch.load(PATH))
net.eval()

Using cache found in /home/lcarranza/.cache/torch/hub/pytorch_vision_v0.5.0


Let's use 2 GPUs!


Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, t

In [23]:
outputs = net(images)

In [24]:
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % testset.idx_to_class[predicted[j].item()]
                              for j in range(4)))

Predicted:  jamon nueces caramelo  jugo


In [25]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        # images, labels = data
        images, labels = data[0].to(device), data[1].to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 63 %


In [26]:
class_correct = list(0. for i in range(25))
class_total = list(0. for i in range(25))
with torch.no_grad():
    for data in testloader:
        # images, labels = data
        images, labels = data[0].to(device), data[1].to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(len(c)):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(25):
    print('Accuracy of %5s : %2d %%' % (
        trainset.idx_to_class[i], 100 * class_correct[i] / class_total[i]))

Accuracy of aceite : 66 %
Accuracy of  agua : 70 %
Accuracy of arroz : 44 %
Accuracy of azucar : 71 %
Accuracy of  cafe : 57 %
Accuracy of caramelo : 66 %
Accuracy of cereal : 50 %
Accuracy of chips : 45 %
Accuracy of chocolate : 63 %
Accuracy of especias : 46 %
Accuracy of frijoles : 62 %
Accuracy of gaseosa : 81 %
Accuracy of harina : 71 %
Accuracy of jamon : 73 %
Accuracy of  jugo : 63 %
Accuracy of leche : 60 %
Accuracy of  maiz : 83 %
Accuracy of  miel : 75 %
Accuracy of nueces : 50 %
Accuracy of pasta : 72 %
Accuracy of pescado : 85 %
Accuracy of salsatomate : 90 %
Accuracy of    te : 50 %
Accuracy of torta : 50 %
Accuracy of vinagre : 50 %


In [ ]:
Era 39%
Era 46%
Era 51% 200
Sigue 51% 400
Es 63% 734